# Genomics & Variant Effect Prediction — Starter Notebook

In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from joblib import dump

# Point to your CSV (demo or ClinVar-cleaned)
DATA_PATH = Path("../data/demo_variants.csv")  # replace with ../data/clinvar_clean.csv when ready
LABEL_COL = "clinical_significance"
POS_LABEL = "Pathogenic"
NEG_LABEL = "Benign"

df = pd.read_csv(DATA_PATH)

# If the label column has a different name, try to normalize it
if LABEL_COL not in df.columns:
    for c in ["ClinicalSignificance","clinical_significance_norm","clinicalsignificance","clinsig","CLNSIG"]:
        if c in df.columns:
            df = df.rename(columns={c: LABEL_COL})
            break

# Normalize label values if needed
def _norm(s):
    s = str(s).lower().replace(" ", "_")
    if "pathogenic" in s:   # includes likely_pathogenic
        return "Pathogenic"
    if "benign" in s:       # includes likely_benign
        return "Benign"
    return None

df[LABEL_COL] = df[LABEL_COL].map(_norm)
df = df[df[LABEL_COL].isin([POS_LABEL, NEG_LABEL])].copy()

print("Head:\n", df.head(), "\n")
print("Label counts:\n", df[LABEL_COL].value_counts(), "\n")

# Ensure required numeric features exist
for col in ["grantham","is_conserved","in_domain","hydrophobicity_delta","charge_delta"]:
    if col not in df.columns:
        df[col] = 0.0

X = df[["grantham","is_conserved","in_domain","hydrophobicity_delta","charge_delta"]].copy()
y = (df[LABEL_COL] == POS_LABEL).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Shapes:", X_train.shape, X_test.shape, " PosRate(test)=", y_test.mean())


In [ ]:

# Build models
logit = Pipeline([("scaler", StandardScaler()),
                  ("clf", LogisticRegression(max_iter=200, class_weight="balanced"))])

rf = Pipeline([("clf", RandomForestClassifier(n_estimators=300, class_weight="balanced", random_state=42))])

models = {"LogisticRegression": logit, "RandomForest": rf}
metrics = {}
for name, m in models.items():
    m.fit(X_train, y_train)
    proba = m.predict_proba(X_test)[:,1]
    metrics[name] = {
        "roc_auc": roc_auc_score(y_test, proba),
        "pr_auc": average_precision_score(y_test, proba)
    }
metrics


In [ ]:

# Plot ROC and PR for each model (matplotlib only, one plot per figure, no explicit colors)
def plot_roc(y_true, y_score, title):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    plt.figure()
    plt.plot(fpr, tpr, label="ROC")
    plt.plot([0,1],[0,1],"--")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate"); plt.title(title); plt.legend(); plt.show()

def plot_pr(y_true, y_score, title):
    p, r, _ = precision_recall_curve(y_true, y_score)
    plt.figure()
    plt.plot(r, p, label="PR")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(title); plt.legend(); plt.show()

for name, m in models.items():
    proba = m.predict_proba(X_test)[:,1]
    plot_roc(y_test, proba, f"{name} ROC")
    plot_pr(y_test, proba, f"{name} PR")


In [ ]:

# Confusion matrices and classification reports @ 0.5
for name, m in models.items():
    proba = m.predict_proba(X_test)[:,1]
    pred = (proba >= 0.5).astype(int)
    print(f"\n{name} Confusion Matrix:\n", confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred, target_names=[NEG_LABEL, POS_LABEL]))


In [ ]:

# Feature importances / coefficients
feat_names = X.columns.tolist()

# Logistic Regression coefficients (after scaling)
m = models["LogisticRegression"].fit(X_train, y_train)
lr = m.named_steps["clf"]
import pandas as pd
lr_coef = pd.DataFrame({"feature": feat_names, "coef": lr.coef_[0]}).sort_values("coef", ascending=False)
print("\nLogistic Regression coefficients:\n", lr_coef)

# RandomForest importances
m = models["RandomForest"].fit(X_train, y_train)
rf = m.named_steps["clf"]
rf_imp = pd.DataFrame({"feature": feat_names, "importance": rf.feature_importances_}).sort_values("importance", ascending=False)
print("\nRandomForest importances:\n", rf_imp)

plt.figure()
plt.bar(rf_imp["feature"], rf_imp["importance"])
plt.title("RandomForest Feature Importances")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:

# Save best model by PR-AUC
best_name = max(metrics, key=lambda k: metrics[k]["pr_auc"])
best_model = models[best_name].fit(X_train, y_train)
MODEL_PATH = Path(f"../model_{best_name.lower()}.joblib")
dump(best_model, MODEL_PATH)
print("Saved best model:", best_name, "->", MODEL_PATH.resolve())

def score_variant(model, features: dict) -> float:
    row = pd.DataFrame([features])[["grantham","is_conserved","in_domain","hydrophobicity_delta","charge_delta"]]
    return float(model.predict_proba(row)[:,1][0])

print("Example score:", score_variant(best_model, {
    "grantham": 120,
    "is_conserved": 1,
    "in_domain": 1,
    "hydrophobicity_delta": 0.2,
    "charge_delta": 1
}))
